In [240]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

def xml_to_csv(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    data = []
    paragrafnummer = 1  # Initialize XML structure
    
    for paragraph in root.findall('.//paragraph'):
        for sentence in paragraph.findall('.//sentence'):
            for word in sentence.findall('.//w'):
                data.append({
                    "paragrafnummer": paragrafnummer,
                    "token": word.text,
                    "msd": word.get('msd', ''),
                    "lemma": word.get('lemma', ''),
                    "lex": word.get('lex', ''),
                    "sense": word.get('sense', ''),
                    "complemgram": word.get('complemgram', ''),
                    "compwf": word.get('compwf', ''),
                    "sentimentclass": word.get('sentimentclass', ''),
                    "ref": word.get('ref', ''),
                    "dephead": word.get('dephead', ''),
                    "deprel": word.get('deprel', '')
                })
        paragrafnummer += 1  # Increment for each paragraph
    
    df = pd.DataFrame(data)
    df['dephead'] = df['dephead'].replace("", 0)
    # Convert sentimentclass values to numerical values
    sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1, "": ""}
    df['sentimentclass'] = df['sentimentclass'].map(sentiment_mapping)

    csv_path = xml_path.replace('.xml', '.csv')  # Or specify your own path
    df.to_csv(csv_path, index=False)
    return csv_path



# Use the function
xml_path = 'testtestdata.xml'
csv_path = xml_to_csv(xml_path)
print(f"CSV file created at: {csv_path}")


CSV file created at: testtestdata.csv


In [241]:
testtestdata_df= pd.read_csv(csv_path, encoding='utf-8')
testtestorg_df=pd.read_csv('testtestorg.csv', header=None, names=['paragrafnummer', 'sentence', 'category'], encoding='utf-8')

In [242]:
testtestdata_df

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel
0,1,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0.0,2,0,ROOT
2,1,upplyft,PC.PRF.UTR.SIN.IND.NOM,|upplyft|,|upplyft..av.1|,|upplyft..1:-1.000|,|upp..ab.1+lyfta..vb.1:6.154e-10|,|upp+lyft|,NaN,3,2,SP
3,1,av,PP,|av|,|av..pp.1|,|av..1:-1.000|,|,|,NaN,4,2,AG
4,1,det,DT.NEU.SIN.DEF,|en|den|,|en..al.1|den..pn.1|,|den..1:-1.000|en..2:-1.000|den..2:-1.000|,|,|,NaN,5,6,DT
...,...,...,...,...,...,...,...,...,...,...,...,...
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP


In [243]:


# Add paragrafnummer from testtestorg to testtestdata
testtestdata_df['sentimentval'] = testtestdata_df['paragrafnummer'].apply(
    lambda x: testtestorg_df.at[x-1, 'paragrafnummer'] if x-1 < len(testtestorg_df) else None
)

# Map the category from testtestorg to testtestdata
testtestdata_df['category'] = testtestdata_df['paragrafnummer'].apply(
    lambda x: testtestorg_df.at[x-1, 'category'] if x-1 < len(testtestorg_df) else None
)

# Replace âˆ’ with - in the entire DataFrame
testtestdata_df = testtestdata_df.replace('−', '-', regex=True)

# Save the final corrected DataFrame to a new CSV file
final_csv_path = 'final_corrected_testtestdata.csv'
testtestdata_df.to_csv(final_csv_path, index=False)

print(f"Final corrected CSV saved to: {final_csv_path}")


Final corrected CSV saved to: final_corrected_testtestdata.csv


In [244]:
df= pd.read_csv('final_corrected_testtestdata.csv', encoding='utf-8')
unique_values = df['sentimentval'].unique()
print(unique_values)
df = df['dephead'].fillna(0)


[ 1  0 -1]


In [245]:
# Read the CSV file into a DataFrame
predictDf = pd.read_csv('final_corrected_testtestdata.csv', encoding='utf-8')
predictDf = predictDf.dropna()

# Convert sentimentclass values to numerical values
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
predictDf['sentimentclass'] = predictDf['sentimentclass'].map(sentiment_mapping)

# Group by paragrafnummer and calculate the average sentimentclass
average_sentiment = predictDf.groupby('paragrafnummer')['sentimentclass'].mean()



predictDf['average_sentiment'] = average_sentiment
predictDf

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category,average_sentiment
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,NaN,2,0,ROOT,1,egna meningar,NaN
5,1,stöd,NN.NEU.SIN.IND.NOM,|stöd|,|stöd..nn.1|,|stöd..1:0.821|stöd..2:0.179|,|,|,NaN,6,4,PA,1,egna meningar,NaN
7,1,fick,VB.PRT.AKT,|få|,|få..vb.1|,|få..1:0.942|få..3:0.058|,|,|,NaN,8,6,ET,1,egna meningar,NaN
10,1,kollegor,NN.UTR.PLU.IND.NOM,|kollega|,|kollega..nn.1|,|kollega..1:-1.000|,|koll..nn.1+lega..nn.1:3.860e-13|koll..nn.2+le...,|koll+legor|kol+legor|,NaN,11,9,PA,1,egna meningar,NaN
13,2,nya,JJ.POS.UTR+NEU.SIN.DEF.NOM,|ny|,|ny..av.1|,|ny..1:-1.000|,|,|,NaN,2,3,AT,1,egna meningar,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,169,järnvägar,NN.UTR.PLU.IND.NOM,|järnväg|,|järnväg..nn.1|,|järnväg..1:-1.000|,|järn..nn.1+väg..nn.1:1.015e-10|,|järn+vägar|,NaN,14,13,CJ,0,8 Sidor,NaN
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,NaN,2,0,ROOT,0,8 Sidor,NaN
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,NaN,4,2,OO,0,8 Sidor,NaN
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,NaN,2,0,ROOT,-1,egna meningar,NaN


In [246]:
# Read the CSV file into a DataFrame
predictDf = pd.read_csv('final_corrected_testtestdata.csv', encoding='utf-8')
predictDf = predictDf.dropna()

# Convert sentimentclass values to numerical values
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
predictDf['sentimentclass'] = predictDf['sentimentclass'].map(sentiment_mapping)

# Group by paragrafnummer and calculate the average sentimentclass
average_sentiment = predictDf.groupby('paragrafnummer')['sentimentclass'].transform('mean')

# Assign the average sentiment to each row
predictDf['average_sentiment'] = average_sentiment

# Fill NaN values in case there are paragraphs without sentimentclass
predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')

predictDf

C:\Users\ernil\AppData\Local\Temp\ipykernel_16840\116262771.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')


,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category,average_sentiment
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,NaN,2,0,ROOT,1,egna meningar,NaN
5,1,stöd,NN.NEU.SIN.IND.NOM,|stöd|,|stöd..nn.1|,|stöd..1:0.821|stöd..2:0.179|,|,|,NaN,6,4,PA,1,egna meningar,NaN
7,1,fick,VB.PRT.AKT,|få|,|få..vb.1|,|få..1:0.942|få..3:0.058|,|,|,NaN,8,6,ET,1,egna meningar,NaN
10,1,kollegor,NN.UTR.PLU.IND.NOM,|kollega|,|kollega..nn.1|,|kollega..1:-1.000|,|koll..nn.1+lega..nn.1:3.860e-13|koll..nn.2+le...,|koll+legor|kol+legor|,NaN,11,9,PA,1,egna meningar,NaN
13,2,nya,JJ.POS.UTR+NEU.SIN.DEF.NOM,|ny|,|ny..av.1|,|ny..1:-1.000|,|,|,NaN,2,3,AT,1,egna meningar,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,169,järnvägar,NN.UTR.PLU.IND.NOM,|järnväg|,|järnväg..nn.1|,|järnväg..1:-1.000|,|järn..nn.1+väg..nn.1:1.015e-10|,|järn+vägar|,NaN,14,13,CJ,0,8 Sidor,NaN
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,NaN,2,0,ROOT,0,8 Sidor,NaN
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,NaN,4,2,OO,0,8 Sidor,NaN
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,NaN,2,0,ROOT,-1,egna meningar,NaN


In [247]:
lista_negationer = ["inte", "ej", "icke", "ingen", "inget", "inga", "knappast", "aldrig", "ingenting", "ingenstans", "sällan"]

In [248]:
testingtest = pd.read_csv('final_corrected_testtestdata.csv', encoding='utf-8')

In [249]:
testingtest

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category
0,1,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS,1,egna meningar
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0.0,2,0,ROOT,1,egna meningar
2,1,upplyft,PC.PRF.UTR.SIN.IND.NOM,|upplyft|,|upplyft..av.1|,|upplyft..1:-1.000|,|upp..ab.1+lyfta..vb.1:6.154e-10|,|upp+lyft|,NaN,3,2,SP,1,egna meningar
3,1,av,PP,|av|,|av..pp.1|,|av..1:-1.000|,|,|,NaN,4,2,AG,1,egna meningar
4,1,det,DT.NEU.SIN.DEF,|en|den|,|en..al.1|den..pn.1|,|den..1:-1.000|en..2:-1.000|den..2:-1.000|,|,|,NaN,5,6,DT,1,egna meningar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT,-1,egna meningar
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS,-1,egna meningar
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN,-1,egna meningar
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP,-1,egna meningar


In [250]:
def negation_score(dataF, negList) :
    result = dataF.copy()  # Create a copy of the DataFrame to avoid modifying the original
    for index, row in result.iterrows():
        if row['token'] in negList:
            for index2, row2 in result.iterrows():
                if row2['ref'] == row['dephead']:
                    result.loc[index2, 'sentimentclass'] *= -1
            # Modify the row or do any other operation here
            print(f"Found negation word '{row['token']}' in row {index}")

    return result

negation_score(testingtest, lista_negationer)

Found negation word 'inte' in row 203
Found negation word 'inte' in row 256
Found negation word 'inte' in row 292
Found negation word 'ingenting' in row 548
Found negation word 'inte' in row 1032
Found negation word 'inte' in row 1065
Found negation word 'inte' in row 1086
Found negation word 'inte' in row 1113
Found negation word 'inte' in row 1203
Found negation word 'inte' in row 1240
Found negation word 'inte' in row 1314
Found negation word 'inte' in row 1397
Found negation word 'ingen' in row 1428
Found negation word 'inte' in row 1458
Found negation word 'inte' in row 1526
Found negation word 'inga' in row 1602
Found negation word 'inte' in row 1608


,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category
0,1,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS,1,egna meningar
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0.0,2,0,ROOT,1,egna meningar
2,1,upplyft,PC.PRF.UTR.SIN.IND.NOM,|upplyft|,|upplyft..av.1|,|upplyft..1:-1.000|,|upp..ab.1+lyfta..vb.1:6.154e-10|,|upp+lyft|,NaN,3,2,SP,1,egna meningar
3,1,av,PP,|av|,|av..pp.1|,|av..1:-1.000|,|,|,NaN,4,2,AG,1,egna meningar
4,1,det,DT.NEU.SIN.DEF,|en|den|,|en..al.1|den..pn.1|,|den..1:-1.000|en..2:-1.000|den..2:-1.000|,|,|,NaN,5,6,DT,1,egna meningar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT,-1,egna meningar
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS,-1,egna meningar
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN,-1,egna meningar
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP,-1,egna meningar


In [251]:
def negation_score_optimized(dataF, negList):
    # Create a copy of the DataFrame to avoid modifying the original
    result = dataF.copy()
    
    # Find indices of negation words
    neg_indices = result[result['token'].isin(negList)].index
    
    # Map each negation word to its dependent sentence or token
    for neg_index in neg_indices:
        dep_head = result.at[neg_index, 'dephead']
        
        # Find all tokens that are dependent on this negation word
        dependent_indices = result[result['ref'] == dep_head].index
        
        # Invert sentiment class for all dependents
        result.loc[dependent_indices, 'sentimentclass'] *= -1
        
        # Optional: Print statement to indicate operation
        print(f"Found negation word '{result.at[neg_index, 'token']}' in row {neg_index}")

    return result

# Example usage
# negation_score_optimized(testingtest, lista_negationer)
negation_score_optimized(testingtest, lista_negationer).tail(11)


Found negation word 'inte' in row 203
Found negation word 'inte' in row 256
Found negation word 'inte' in row 292
Found negation word 'ingenting' in row 548
Found negation word 'inte' in row 1032
Found negation word 'inte' in row 1065
Found negation word 'inte' in row 1086
Found negation word 'inte' in row 1113
Found negation word 'inte' in row 1203
Found negation word 'inte' in row 1240
Found negation word 'inte' in row 1314
Found negation word 'inte' in row 1397
Found negation word 'ingen' in row 1428
Found negation word 'inte' in row 1458
Found negation word 'inte' in row 1526
Found negation word 'inga' in row 1602
Found negation word 'inte' in row 1608


,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category
1600,170,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS,0,8 Sidor
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,0.0,2,0,ROOT,0,8 Sidor
1602,170,inga,DT.UTR+NEU.PLU.IND,|ingen|,|ingen..pn.1|,|ingen..1:-1.000|,|,|,NaN,3,4,DT,0,8 Sidor
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,-1.0,4,2,OO,0,8 Sidor
1604,170,.,MAD,|,|,|,|,|,NaN,5,2,IP,0,8 Sidor
1605,171,Idag,AB,|idag|,|idag..ab.1|,|i_dag..1:-1.000|i_dag..2:-1.000|,|,|,NaN,1,2,TA,-1,egna meningar
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT,-1,egna meningar
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS,-1,egna meningar
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN,-1,egna meningar
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP,-1,egna meningar


In [252]:
def apply_negation_by_paragraph(df, negList):
    # Apply the negation score function to each group and combine the results
    result_df = df.groupby('paragrafnummer').apply(negation_score_optimized, negList=negList)
    
    # Reset index if necessary (depends on your negation function)
    result_df = result_df.reset_index(drop=True)
    
    return result_df

In [253]:
# Read the CSV file into a DataFrame
predictDf = pd.read_csv('final_corrected_testtestdata.csv')
predictDf = predictDf.dropna()

# # Convert sentimentclass values to numerical values
# sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
# predictDf['sentimentclass'] = predictDf['sentimentclass'].map(sentiment_mapping)

# Group by paragrafnummer and calculate the average sentimentclass
average_sentiment = predictDf.groupby('paragrafnummer')['sentimentclass'].transform('mean')

# Assign the average sentiment to each row
predictDf['average_sentiment'] = average_sentiment

# Fill NaN values in case there are paragraphs without sentimentclass
predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')

predictDf

C:\Users\ernil\AppData\Local\Temp\ipykernel_16840\2937337475.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')


,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category,average_sentiment
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0.0,2,0,ROOT,1,egna meningar,0.500000
5,1,stöd,NN.NEU.SIN.IND.NOM,|stöd|,|stöd..nn.1|,|stöd..1:0.821|stöd..2:0.179|,|,|,1.0,6,4,PA,1,egna meningar,0.500000
7,1,fick,VB.PRT.AKT,|få|,|få..vb.1|,|få..1:0.942|få..3:0.058|,|,|,1.0,8,6,ET,1,egna meningar,0.500000
10,1,kollegor,NN.UTR.PLU.IND.NOM,|kollega|,|kollega..nn.1|,|kollega..1:-1.000|,|koll..nn.1+lega..nn.1:3.860e-13|koll..nn.2+le...,|koll+legor|kol+legor|,0.0,11,9,PA,1,egna meningar,0.500000
13,2,nya,JJ.POS.UTR+NEU.SIN.DEF.NOM,|ny|,|ny..av.1|,|ny..1:-1.000|,|,|,0.0,2,3,AT,1,egna meningar,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,169,järnvägar,NN.UTR.PLU.IND.NOM,|järnväg|,|järnväg..nn.1|,|järnväg..1:-1.000|,|järn..nn.1+väg..nn.1:1.015e-10|,|järn+vägar|,0.0,14,13,CJ,0,8 Sidor,0.000000
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,0.0,2,0,ROOT,0,8 Sidor,0.500000
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,1.0,4,2,OO,0,8 Sidor,0.500000
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT,-1,egna meningar,0.500000


In [254]:
# def negation_score_optimized(group, negList):
#     # Implement the logic to modify the sentiment scores based on negations within the group
#     # This is a placeholder for your actual negation handling logic
#     for index, row in group.iterrows():
#         if row['token'] in negList:
#             # This is a simplified example of what you might do:
#             # You could invert the sentiment score of sentences or tokens that are related
#             group.loc[index, 'sentimentclass'] *= -1
#     return group

# Apply negation handling logic to each paragraph group in the DataFrame
def apply_negation_by_paragraph(df, negList):
    # Apply the negation score function to each group and combine the results
    result_df = df.groupby('paragrafnummer', group_keys=False).apply(negation_score_optimized, negList=negList)
    return result_df

tttt = apply_negation_by_paragraph(testingtest, lista_negationer)

Found negation word 'inte' in row 203
Found negation word 'inte' in row 256
Found negation word 'inte' in row 292
Found negation word 'ingenting' in row 548
Found negation word 'inte' in row 1032
Found negation word 'inte' in row 1065
Found negation word 'inte' in row 1086
Found negation word 'inte' in row 1113
Found negation word 'inte' in row 1203
Found negation word 'inte' in row 1240
Found negation word 'inte' in row 1314
Found negation word 'inte' in row 1397
Found negation word 'ingen' in row 1428
Found negation word 'inte' in row 1458
Found negation word 'inte' in row 1526
Found negation word 'inga' in row 1602
Found negation word 'inte' in row 1608


In [255]:
testingtest.tail(11)

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category
1600,170,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS,0,8 Sidor
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,0.0,2,0,ROOT,0,8 Sidor
1602,170,inga,DT.UTR+NEU.PLU.IND,|ingen|,|ingen..pn.1|,|ingen..1:-1.000|,|,|,NaN,3,4,DT,0,8 Sidor
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,1.0,4,2,OO,0,8 Sidor
1604,170,.,MAD,|,|,|,|,|,NaN,5,2,IP,0,8 Sidor
1605,171,Idag,AB,|idag|,|idag..ab.1|,|i_dag..1:-1.000|i_dag..2:-1.000|,|,|,NaN,1,2,TA,-1,egna meningar
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,0.0,2,0,ROOT,-1,egna meningar
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS,-1,egna meningar
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN,-1,egna meningar
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP,-1,egna meningar


In [256]:
tttt.tail(11)

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,ref,dephead,deprel,sentimentval,category
1600,170,Jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,1,2,SS,0,8 Sidor
1601,170,har,VB.PRS.AKT,|ha|,|ha..vb.1|,|ha..1:0.745|ha..3:0.255|,|,|,0.0,2,0,ROOT,0,8 Sidor
1602,170,inga,DT.UTR+NEU.PLU.IND,|ingen|,|ingen..pn.1|,|ingen..1:-1.000|,|,|,NaN,3,4,DT,0,8 Sidor
1603,170,vänner,NN.UTR.PLU.IND.NOM,|vän|,|vän..nn.1|,|vän..1:-1.000|,|,|,-1.0,4,2,OO,0,8 Sidor
1604,170,.,MAD,|,|,|,|,|,NaN,5,2,IP,0,8 Sidor
1605,171,Idag,AB,|idag|,|idag..ab.1|,|i_dag..1:-1.000|i_dag..2:-1.000|,|,|,NaN,1,2,TA,-1,egna meningar
1606,171,är,VB.PRS.AKT,|vara|,|vara..vb.1|,|vara..1:-1.000|,|,|,-0.0,2,0,ROOT,-1,egna meningar
1607,171,jag,PN.UTR.SIN.DEF.SUB,|jag|,|jag..pn.1|,|jag..1:-1.000|,|,|,NaN,3,2,SS,-1,egna meningar
1608,171,inte,AB,|inte|,|inte..ab.1|,|inte..1:-1.000|,|,|,NaN,4,2,NaN,-1,egna meningar
1609,171,glad,JJ.POS.UTR.SIN.IND.NOM,|glad|,|glad..av.1|,|glad..1:-1.000|,|,|,1.0,5,2,SP,-1,egna meningar
